In [8]:
import polars as pl

In [32]:
filename = "./data/tick/NQ 09_23.20230818.parquet"

In [33]:
(pl.scan_parquet(filename)
    .fetch(10)
)

FileNotFoundError: No such file or directory: ./data/tick/NQ 09_23.20230818.parquet

In [31]:
!dir .\\data

 Volume in drive F is TS1
 Volume Serial Number is B4DA-75AE

 Directory of F:\Chris\workspace\Notebooks\data\tick



File Not Found
